In [1]:
import os
from pymilvus import MilvusClient
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Ollama
from langchain import hub
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus
from langchain import hub
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import requests
import sys
import io
import json
import shutil
import sys
import datetime
import subprocess
import math
import base64
from time import gmtime, strftime
import random, string
import time
import base64
import uuid
import socket
from pymilvus import connections
from pymilvus import utility
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
from pymilvus import MilvusClient
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from dotenv import load_dotenv
load_dotenv(verbose=True)
from datetime import datetime, timedelta
from langchain_milvus import Milvus

DIMENSION = 384 
TOKEN = os.environ.get("ZILLIZ_TOKEN")
MILVUS_URL = os.environ.get("SERVERLESS_VECTORDB_URI")

slack_token = os.environ.get("SLACK_BOT_TOKEN")
client = WebClient(token=slack_token)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

COLLECTION_NAME = "liveplanes"
TODAYS_DATE = str( datetime.today().strftime('%Y-%m-%d') ) 
YESTERDAYS_DATE = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')

os.environ["LANGCHAIN_HUB_API_URL"] = "https://api.hub.langchain.com"
os.environ["LANGCHAIN_HUB_API_KEY"] = os.environ.get("LANGCHAIN_HUB_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.environ.get("LANGCHAIN_HUB_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.hub.langchain.com"
os.environ["TOKENIZERS_PARALLELISM"]  = "true"

### Turn off slack warnings
os.environ["SKIP_SLACK_SDK_WARNING"] = "false"


/Users/timothyspann/Downloads/code/milvusvenv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/timothyspann/Downloads/code/milvusvenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
# -----------------------------------------------------------------------------
# Connect to Milvus

vector_store = Milvus(
    embedding_function=embeddings,
    collection_name=COLLECTION_NAME,
    primary_field = "id",
    text_field="details",
    vector_field = "plane_text_vector",
    connection_args={"uri": MILVUS_URL, "token": TOKEN},
)

In [3]:
def run_query() -> None:
    llm = Ollama(
        model="llama3.1",
        callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
        stop=["<|eot_id|>"],
    )

    query = input("\nQuery: ")
    qa_chain = RetrievalQA.from_chain_type(
        llm, retriever=vector_store.as_retriever(collection =  COLLECTION_NAME))


    # , expr="parameter == 'o3'"
    result = qa_chain.invoke({"query": query})
    
    resultforslack = str(result["result"])
    print(resultforslack)

    try:
        response = client.chat_postMessage(mrkdwn=True, channel="C06NE1FU6SE", text="", 
                                            blocks=[{"type": "section","text": {"type": "mrkdwn","text": "*" + str(query) + "*  \n\n" + str(resultforslack) +"\n" }}])

    except SlackApiError as e:
        # You will get a SlackApiError if "ok" is False
        print("Slack failed")

/var/folders/c2/6wq9klzn19d97gvw8f8j1jdr0000gn/T/ipykernel_82902/3689434376.py:2: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(



Query:  What are five Boeing from American Airlines flights?


I don't know. The provided context only mentions two flights of American Airlines, AAL1056 and one other flight (NA) but the aircraft type is not mentioned, it could be a Boeing or another manufacturer.I don't know. The provided context only mentions two flights of American Airlines, AAL1056 and one other flight (NA) but the aircraft type is not mentioned, it could be a Boeing or another manufacturer.


/var/folders/c2/6wq9klzn19d97gvw8f8j1jdr0000gn/T/ipykernel_82902/3689434376.py:2: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(



Query:  Describe everything you know about Boeing from American Airlines 


...

Here's what I can gather from the provided context:

**Flight Information**

* Flight number: AAL1056
* Aircraft type: B738 (Boeing 737-800)
* Operator: American Airlines
* Country of origin: United States

**Location and Altitude**

* Location: approximately at 40.620071 latitude and -74.554321 longitude
* Altitude:
	+ Barometric altitude: 19,125 feet
	+ Geodetic altitude: 19,725 feet

**Speed and Time**

* Speed: 414.6 mph (mph = miles per hour)
* Date and time: September 22, 2024, at 15:44:30

I don't know any additional information beyond what's provided in this context.Here's what I can gather from the provided context:

**Flight Information**

* Flight number: AAL1056
* Aircraft type: B738 (Boeing 737-800)
* Operator: American Airlines
* Country of origin: United States

**Location and Altitude**

* Location: approximately at 40.620071 latitude and -74.554321 longitude
* Altitude:
	+ Barometric altitude: 19,125 feet
	+ Geodetic altitude: 19,725 feet

**Speed and Time**

* Speed: 

/var/folders/c2/6wq9klzn19d97gvw8f8j1jdr0000gn/T/ipykernel_82902/3689434376.py:2: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(



Query:  Tell me everything about UAL2072


Based on the provided context, here's everything I know about UAL2072:

* Aircraft type: B753 (built by Boeing)
* Operator: United Airlines (from the United States)
* Flight number: UAL2072
* Speed: 171.8 mph at 2024-09-22 15:44:38
* Location: Latitude 40.317357, longitude -74.417641
* Altitude:
	+ Barometric altitude: 2925 feet
	+ Geodetic altitude: 2950 feetBased on the provided context, here's everything I know about UAL2072:

* Aircraft type: B753 (built by Boeing)
* Operator: United Airlines (from the United States)
* Flight number: UAL2072
* Speed: 171.8 mph at 2024-09-22 15:44:38
* Location: Latitude 40.317357, longitude -74.417641
* Altitude:
	+ Barometric altitude: 2925 feet
	+ Geodetic altitude: 2950 feet


/var/folders/c2/6wq9klzn19d97gvw8f8j1jdr0000gn/T/ipykernel_82902/3689434376.py:2: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(
